# 1 - Introducción



# 2 - Resumnen de secciones:

- Breve descripción de lo que hará el notebook:
- Cargar el dataset
- Hacer limpieza básica
- Preparar texto combinado
- Seleccionar una muestra
- Generar embeddings con OpenAI
- Crear un recomendador basado en similitud

# 3 - Cargar dataset
- Leer archivos desde Google Drive

Comentarios:

- Aquí simplemente cargamos los tres archivos que vamos a usar en el prototipo.

- En el caso de credits.csv y keywords.csv, agregamos encoding="utf-8" porque suelen traer caracteres especiales y Colab a veces da error si no se especifica.

- Las impresiones rápidas (print) nos ayudan a verificar que los archivos tienen el número esperado de filas y columnas antes de continuar.

In [1]:
import pandas as pd
import numpy as np
import ast

# Rutas directas porque los archivos están en la misma carpeta del Colab
movies_path = "movies_metadata.csv"
credits_path = "credits.csv"
keywords_path = "keywords.csv"

# Carga de los CSV
movies_df = pd.read_csv(movies_path, low_memory=False)
credits_df = pd.read_csv(credits_path, encoding="utf-8")
keywords_df = pd.read_csv(keywords_path, encoding="utf-8")

# Verificamos que todo se haya cargado correctamente
print("movies_df:", movies_df.shape)
print("credits_df:", credits_df.shape)
print("keywords_df:", keywords_df.shape)

movies_df.head()


movies_df: (45466, 24)
credits_df: (45476, 3)
keywords_df: (46419, 2)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# 4 - Limpieza de datos (primer bloque de código real)

Esta es la sección que tú pediste que vaya primero en código.

Incluye:

- Manejo de valores nulos
- Corrección de tipos
- Parseo de columnas JSON (solo las necesarias)
- Eliminación de filas sin overview
- Selección de columnas relevantes



Esta limpieza es muy simple pero suficiente para un prototipo.

- No hacemos nada complejo: solo nos aseguramos de que la descripción esté presente porque es lo que usaremos para embeddings.

- Reparamos el ID porque lo necesitaremos para unir los dataframes más adelante.

In [2]:
# Conversión básica de tipos y limpieza necesaria para el prototipo

# Convertimos la fecha a datetime. Si falla, lo dejamos como NaT.
movies_df["release_date"] = pd.to_datetime(movies_df["release_date"], errors="coerce")

# Quitamos filas sin overview porque no sirven para generar embeddings
movies_df = movies_df.dropna(subset=["overview"])

# Nos aseguramos de que la columna 'id' tenga valores válidos
movies_df = movies_df[movies_df["id"].notna()]

# Convertimos la columna id a numérico. Las filas que no se puedan convertir se descartan.
movies_df["id"] = pd.to_numeric(movies_df["id"], errors="coerce")
movies_df = movies_df.dropna(subset=["id"])

# Convertimos a entero
movies_df["id"] = movies_df["id"].astype(int)

# Vista rápida para confirmar cambios
movies_df.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# 5 - Preparación del texto combinado

Uniremos:
- overview
- genres
- keywords
- cast
- crew (solo director)

Ese texto se usará para generar embeddings.

Comentarios:
- estas columnas parecen listas o diccionarios, pero realmente son texto plano.
- ast.literal_eval() convierte ese texto a estructuras Python reales.
- Si algún valor está roto, devolvemos una lista vacía para evitar errores.

In [3]:
# Varias columnas vienen como strings con formato JSON.
# Aquí las convertimos a estructuras reales de Python para poder usarlas.

import ast

def parse_json_column(value):
    try:
        return ast.literal_eval(value)
    except:
        return []

# Parseamos 'genres' en movies_df
movies_df["genres"] = movies_df["genres"].apply(parse_json_column)

# Parseamos 'keywords' en keywords_df
keywords_df["keywords"] = keywords_df["keywords"].apply(parse_json_column)

# Parseamos cast y crew en credits_df
credits_df["cast"] = credits_df["cast"].apply(parse_json_column)
credits_df["crew"] = credits_df["crew"].apply(parse_json_column)

# Vista rápida de una columna parseada
movies_df["genres"].head()


,genres
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,"[{'id': 35, 'name': 'Comedy'}]"


Comentarios:
- Hacemos el merge con how="left" porque queremos conservar todas las películas, aunque falten datos en credits o keywords.
- Si una película no tiene keywords, no es problema: ponemos una lista vacía y seguimos.
- El dataframe movies_clean contiene únicamente lo necesario para construir el texto que mandaremos a OpenAI para generar embeddings.

In [4]:
# Hacemos merge para unir toda la información necesaria usando la columna 'id'
movies_df = movies_df.merge(credits_df, on="id", how="left")
movies_df = movies_df.merge(keywords_df, on="id", how="left")

# Si alguna película no tiene keywords, dejamos una lista vacía
movies_df["keywords"] = movies_df["keywords"].apply(lambda x: x if isinstance(x, list) else [])

# Creamos un dataframe reducido con solo las columnas que vamos a usar para generar el texto final
movies_clean = movies_df[["id", "title", "overview", "genres", "keywords", "cast", "crew"]]

# Vista rápida
movies_clean.head()


,id,title,overview,genres,keywords,cast,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


Comentarios:
- Extraemos solo lo que aporta significado.
- No ocupamos nombres de todo el cast porque eso agrega ruido.
- El texto final debe ser claro y representativo del contenido de la película.
- Esta parte define directamente la calidad de las recomendaciones.

In [5]:
# Función para extraer solo los nombres de las listas de JSON
def extract_names(json_list):
    try:
        return " ".join([item["name"] for item in json_list if "name" in item])
    except:
        return ""

# Extraemos géneros
movies_clean["genres_text"] = movies_clean["genres"].apply(extract_names)

# Extraemos keywords
movies_clean["keywords_text"] = movies_clean["keywords"].apply(extract_names)

# Del cast solo tomamos los primeros 3 actores
def extract_main_cast(cast_list):
    try:
        return " ".join([item["name"] for item in cast_list[:3]])
    except:
        return ""

movies_clean["cast_text"] = movies_clean["cast"].apply(extract_main_cast)

# Del crew extraemos solo al director
def extract_director(crew_list):
    try:
        for member in crew_list:
            if member.get("job") == "Director":
                return member["name"]
        return ""
    except:
        return ""

movies_clean["director_text"] = movies_clean["crew"].apply(extract_director)

# Ahora creamos el texto combinado
movies_clean["combined_text"] = (
    movies_clean["overview"].fillna("") + " " +
    movies_clean["genres_text"].fillna("") + " " +
    movies_clean["keywords_text"].fillna("") + " " +
    movies_clean["cast_text"].fillna("") + " " +
    movies_clean["director_text"].fillna("")
)

# Revisamos un ejemplo
movies_clean[["title", "combined_text"]].head()


/tmp/ipython-input-2630023637.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_clean["genres_text"] = movies_clean["genres"].apply(extract_names)
/tmp/ipython-input-2630023637.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_clean["keywords_text"] = movies_clean["keywords"].apply(extract_names)
/tmp/ipython-input-2630023637.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,title,combined_text
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


Comentarios:
- Ahora no colocamos la API key dentro del notebook.
- El archivo .env es detectado con load_dotenv().
- Si .env no está o no tiene la variable correcta, se mostrará un error claro.
- Mucho más seguro y profesional.

In [6]:
!pip install python-dotenv openai --quiet

import os
from dotenv import load_dotenv
from openai import OpenAI

# Cargar el archivo .env
env_path = ".env"
load_dotenv(env_path)

# Obtener la API Key
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("❌ ERROR: La variable OPENAI_API_KEY no se encontró. Revisa tu archivo .env.")

# Crear cliente OpenAI
client = OpenAI(api_key=api_key)

print("🔑 API Key cargada correctamente.")

# ----------------------------
# PRUEBA REAL DE CONEXIÓN
# ----------------------------
try:
    print("Haciendo prueba de conexión a OpenAI...")
    test = client.embeddings.create(
        model="text-embedding-3-small",
        input="test"
    )
    print("✅ Conexión exitosa. Embedding generado correctamente.")
except Exception as e:
    print("❌ ERROR: La API Key no funciona o está mal configurada.")
    print("Mensaje del servidor:", e)


🔑 API Key cargada correctamente.
Haciendo prueba de conexión a OpenAI...
✅ Conexión exitosa. Embedding generado correctamente.


Comentarios:
- sample(n=10000) elige 10,000 filas aleatorias del dataset.
- random_state=42 asegura que la selección sea reproducible.
- Ahora todos los embeddings y cálculos posteriores se harán sobre este subset.
- 10k es una cantidad ideal para probar un recomendador sin costos altos.

In [7]:
# Submuestreamos el dataset para quedarnos solo con 10,000 películas
# Esto acelera el cálculo de embeddings y reduce costos.

movies_clean = movies_clean.sample(n=10000, random_state=42).reset_index(drop=True)

print("Nuevo tamaño del dataset:", len(movies_clean))
movies_clean.head()


Nuevo tamaño del dataset: 10000


,id,title,overview,genres,keywords,cast,crew,genres_text,keywords_text,cast_text,director_text,combined_text
0,292830,Creature,People attend their first halt in a newly open...,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",[],"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5418357ec3a36819a4002e9e', 'de...",Science Fiction Horror,,Bipasha Basu Mukul Dev Imran Abbas Naqvi,Vikram Bhatt,People attend their first halt in a newly open...
1,61752,Girl 6,Girl 6 is a 1996 American film by director Spi...,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 10183, 'name': 'independent film'}, {'...","[{'cast_id': 2, 'character': 'Girl 6', 'credit...","[{'credit_id': '52fe4661c3a368484e08d77b', 'de...",Comedy,independent film neighbor masturbation phone s...,Theresa Randle Isaiah Washington Spike Lee,Spike Lee,Girl 6 is a 1996 American film by director Spi...
2,125229,Samurai Vendetta,Two amiable samurai wind up on opposite sides ...,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",[],"[{'cast_id': 4, 'character': 'Tenge Tanzen', '...","[{'credit_id': '52fe4abfc3a368484e164ba3', 'de...",Drama Action,,Raizô Ichikawa Shintarô Katsu Chitose Maki,Kazuo Mori,Two amiable samurai wind up on opposite sides ...
3,88534,Avé,"While hitchhiking from Sofia to Ruse, Kamen me...","[{'id': 18, 'name': 'Drama'}]","[{'id': 6349, 'name': 'bulgaria'}, {'id': 1313...","[{'cast_id': 2, 'character': 'Ave', 'credit_id...","[{'credit_id': '52fe49f99251416c910bef9f', 'de...",Drama,bulgaria teenager hitchhiking,Anjela Nedyalkova Ovanes Torosian Martin Brambach,Konstantin Bojanov,"While hitchhiking from Sofia to Ruse, Kamen me..."
4,40876,Down to the Bone,A woman stuck in a stale marriage struggles to...,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'id': 242, 'name': 'new york'}, {'id': 1332,...","[{'cast_id': 1, 'character': 'Irene Morrison',...","[{'credit_id': '532d7b54c3a3685fbb001b74', 'de...",Drama Romance,new york cheating cocaine marriage addiction c...,Vera Farmiga Hugh Dillon Clint Jordan,Debra Granik,A woman stuck in a stale marriage struggles to...


# 6 - Generación de embeddings con OpenAI
- text-embedding-3-small


In [8]:
print("Cantidad de registros en movies_clean:", len(movies_clean))

# Qué porcentaje representan respecto al dataset original
original_count = len(movies_df)
clean_count = len(movies_clean)
print(f"Porcentaje del dataset que usaremos: {clean_count/original_count*100:.2f}%")


Cantidad de registros en movies_clean: 10000
Porcentaje del dataset que usaremos: 21.91%


Comentarios:
- Cada texto se recorta si excede un tamaño razonable, porque los embeddings trabajan mejor con inputs controlados.
- Si ocurre un error puntual, no detenemos el proceso completo.
- Asignamos un vector vacío solo en casos excepcionales.
- tqdm nos da la barra de progreso para saber cuánto falta.

In [10]:
from tqdm import tqdm
import time

embeddings = []
texts = movies_clean["combined_text"].tolist()

print("Generando embeddings para 10,000 películas...")

for text in tqdm(texts, desc="Procesando embeddings"):

    # Recortar textos demasiado largos
    if len(text) > 7500:
        text = text[:7500]

    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        vector = response.data[0].embedding
        embeddings.append(vector)

    except Exception:
        # No imprimimos NADA, solo guardamos un vector vacío
        embeddings.append([0] * 1536)
        time.sleep(0.2)

movies_clean["embedding"] = embeddings
movies_clean.head()


Generando embeddings para 10,000 películas...


Procesando embeddings: 100%|██████████| 10000/10000 [31:44<00:00,  5.25it/s]


,id,title,overview,genres,keywords,cast,crew,genres_text,keywords_text,cast_text,director_text,combined_text,embedding
0,292830,Creature,People attend their first halt in a newly open...,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",[],"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5418357ec3a36819a4002e9e', 'de...",Science Fiction Horror,,Bipasha Basu Mukul Dev Imran Abbas Naqvi,Vikram Bhatt,People attend their first halt in a newly open...,"[-0.02787129208445549, 0.03477367386221886, -0..."
1,61752,Girl 6,Girl 6 is a 1996 American film by director Spi...,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 10183, 'name': 'independent film'}, {'...","[{'cast_id': 2, 'character': 'Girl 6', 'credit...","[{'credit_id': '52fe4661c3a368484e08d77b', 'de...",Comedy,independent film neighbor masturbation phone s...,Theresa Randle Isaiah Washington Spike Lee,Spike Lee,Girl 6 is a 1996 American film by director Spi...,"[0.016967305913567543, 0.02504519186913967, -0..."
2,125229,Samurai Vendetta,Two amiable samurai wind up on opposite sides ...,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",[],"[{'cast_id': 4, 'character': 'Tenge Tanzen', '...","[{'credit_id': '52fe4abfc3a368484e164ba3', 'de...",Drama Action,,Raizô Ichikawa Shintarô Katsu Chitose Maki,Kazuo Mori,Two amiable samurai wind up on opposite sides ...,"[-0.019581690430641174, 0.018826348707079887, ..."
3,88534,Avé,"While hitchhiking from Sofia to Ruse, Kamen me...","[{'id': 18, 'name': 'Drama'}]","[{'id': 6349, 'name': 'bulgaria'}, {'id': 1313...","[{'cast_id': 2, 'character': 'Ave', 'credit_id...","[{'credit_id': '52fe49f99251416c910bef9f', 'de...",Drama,bulgaria teenager hitchhiking,Anjela Nedyalkova Ovanes Torosian Martin Brambach,Konstantin Bojanov,"While hitchhiking from Sofia to Ruse, Kamen me...","[-0.017805397510528564, 0.01730971597135067, -..."
4,40876,Down to the Bone,A woman stuck in a stale marriage struggles to...,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'id': 242, 'name': 'new york'}, {'id': 1332,...","[{'cast_id': 1, 'character': 'Irene Morrison',...","[{'credit_id': '532d7b54c3a3685fbb001b74', 'de...",Drama Romance,new york cheating cocaine marriage addiction c...,Vera Farmiga Hugh Dillon Clint Jordan,Debra Granik,A woman stuck in a stale marriage struggles to...,"[-0.008182517252862453, 0.04901142790913582, -..."


# 7 - Cálculo de similitud

- Generamos una matriz o hacemos comparación directa usando cosine similarity.

Comentarios:

- Convertimos los embeddings en una matriz NumPy para acelerar el cálculo.
- Usamos cosine similarity porque mide bien la cercanía semántica.
- Esta matriz es el corazón del recomendador: compara cada película contra todas.

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("Creando la matriz de embeddings...")

# Convertimos la columna "embedding" en una matriz numérica
embedding_matrix = np.vstack(movies_clean["embedding"].values)

print("Calculando matriz de similitud, esto puede tardar unos segundos...")

similarity_matrix = cosine_similarity(embedding_matrix)

print("Matriz de similitud creada con éxito!")


Creando la matriz de embeddings...
Calculando matriz de similitud, esto puede tardar unos segundos...
Matriz de similitud creada con éxito!


# 8 - Función final de recomendación

Comentarios:

- Recuperamos la película buscada usando su título.
- Obtenemos la fila correspondiente de la matriz de similitud.
- Ordenamos por similitud, de mayor a menor.
- Excluimos la película original (primer resultado).
- Retornamos los títulos de las películas más cercana

In [12]:
def recommend(title, n=10):
    # Verificamos si la película existe en el dataset
    if title not in movies_clean["title"].values:
        return f"La película '{title}' no está en el dataset."

    # Obtenemos el índice de la película
    idx = movies_clean[movies_clean["title"] == title].index[0]

    # Obtenemos las películas similares
    similarity_scores = list(enumerate(similarity_matrix[idx]))

    # Ordenamos por score de mayor a menor
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Nos quedamos con los n resultados más similares (excluyendo el mismo título)
    top_movies = similarity_scores[1:n+1]

    # Devolvemos los títulos recomendados
    recommendations = [movies_clean.iloc[i[0]]["title"] for i in top_movies]

    return recommendations


# 9 - Pruebas


Comentarios:
- Probamos con títulos conocidos.
- Si alguno no está en el dataset, el sistema lo indicará.
- Los resultados dependerán completamente de los embeddings generado

In [15]:
# Probamos el sistema de recomendación

print("Probando recomendaciones...\n")

# Puedes reemplazar estos títulos por los que quieras.
test_titles = [
    "Inception",
    "Paranoia"
]

for title in test_titles:
    print(f"\n>>> Recomendaciones para: {title}")
    if title in movies_clean["title"].values:
        recs = recommend(title, n=5)
        print(recs)
    else:
        print("Esta película no está en el dataset.")


Probando recomendaciones...


>>> Recomendaciones para: Inception
['Limitless', 'A Scanner Darkly', 'Paranoia', 'Extracted', 'Hardwired']

>>> Recomendaciones para: Paranoia
['3 Days to Kill', 'Misconduct', 'The Russia House', 'Security', 'Shadow Run']
